In [ ]:
from preprocessed_mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
print(X_train.shape, y_train.shape)
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(X_train[0], cmap="Greys");

In [ ]:
# Build simple NN with 2 hidden layer
# w1.shape=(784, 256) 
# b1.shape=(256,) 
# w2.shape=(256, 10)
# b2.shape=(10,)
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle

# define variables
feature_num = X_train.shape[1] * X_train.shape[2]
digit_num = 10
middle_neuron_num = 512
input_x = tf.placeholder(tf.float32, shape=(None, X_train.shape[1], X_train.shape[2]), name='input_x')
input_x_flatten = tf.reshape(input_x, [-1, feature_num], name='input_x_flatten')
input_y = tf.placeholder(tf.uint8, shape=(None,), name='input_y')
input_y_one_hot = tf.one_hot(input_y, digit_num, name = 'input_y_one_hot') 

w1 = tf.Variable(np.zeros((feature_num, middle_neuron_num), dtype='float32'), name='w1')
b1 = tf.Variable(np.zeros((middle_neuron_num,), dtype='float32'), name='b1')
w2 = tf.Variable(np.random.randn(middle_neuron_num, digit_num) * 0.01, dtype='float32', name='w2')
b2 = tf.Variable(np.zeros((digit_num,), dtype='float32'), name='b2')

# construct NN, use cross-entropy to compute loss function
a1 = tf.nn.sigmoid(tf.matmul(input_x_flatten, w1)+b1, name='a1')
a2 = tf.matmul(a1, w2) + b2
predicted_y_distribution = tf.nn.softmax(a2, name='predicted_y_distribution')
loss = tf.reduce_mean(tf.reduce_sum(-input_y_one_hot*tf.log(predicted_y_distribution), 1), name='loss')
predicted_y = tf.cast(tf.argmax(predicted_y_distribution, axis=1), tf.uint8, name='predicted_y')
correct_prediction = tf.equal(predicted_y, input_y, name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# training
optimizer = tf.train.AdamOptimizer(learning_rate=0.004).minimize(loss, var_list=(w1, b1, w2, b2))
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
tf.train.start_queue_runners()
batch_size = 250
for i in range(4001):
    start_index = i*batch_size % X_train.shape[0]
    end_index = start_index + batch_size
    if start_index == 0: # shuffle training data
        X_train, y_train = shuffle(X_train, y_train)
    x_batch = X_train[start_index:end_index]
    y_batch = y_train[start_index:end_index]
    s.run(optimizer, {input_x: x_batch, input_y: y_batch})
    if i % 100 ==0:
        loss_i = s.run(loss, {input_x: X_train, input_y: y_train})
        accuracy_i = s.run(accuracy, {input_x: X_test, input_y: y_test})
        print("iter %i: loss=%.4f, accuracy on test=%.4f" % (i, loss_i, accuracy_i))
    